In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import os
import re


In [2]:
base_url = "https://www.bank-abc.com"
page_url = f"{base_url}/ar"

headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(page_url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")


In [8]:
bank_name = "Bank ABC"
folder = "bankabc_media"
os.makedirs(folder, exist_ok=True)


In [28]:
slider_items = soup.find_all("div", class_="slider-item")
data = []

In [30]:

for item in slider_items:
    media_url = None
    media_type = "Unknown"

    # ✅ أولاً: نحاول جلب الفيديو
    video_tag = item.find("video")
    if video_tag:
        source = video_tag.find("source")
        if source and source.get("src"):
            media_url = urljoin(base_url, source["src"])
            media_type = "Video"

    # ✅ ثانياً: لو ما فيه فيديو، نحاول نجيب صورة من background-image
    if not media_url:
        bg_div = item.find("div", class_="image-video")
        if bg_div and bg_div.has_attr("style"):
            match = re.search(r'url\([\'"]?(.*?)[\'"]?\)', bg_div["style"])
            if match:
                media_url = urljoin(base_url, match.group(1))
                media_type = "Image"

    # استخراج اسم الملف
    media_name = os.path.basename(media_url.split("?")[0]) if media_url else "No_Media"

    # رابط الخبر
    link_tag = item.find("a", class_="slider-btn")
    article_link = urljoin(base_url, link_tag["href"]) if link_tag and link_tag.get("href") else "No Link"

    # العنوان
    title_tag = item.find("div", class_="slider-item-ttl")
    title = title_tag.get_text(strip=True) if title_tag else "No Title"

    # الوصف المختصر
    desc_tag = item.find("div", class_="slider-small-text")
    description = desc_tag.get_text(strip=True) if desc_tag else "No Description"

    # تحميل الوسيط
    if media_url and media_name != "No_Media":
        try:
            media_data = requests.get(media_url)
            media_data.raise_for_status()
            with open(os.path.join(folder, media_name), "wb") as f:
                f.write(media_data.content)
            print(f"✔️ تم تحميل: {media_name}")
        except Exception as e:
            print(f"❌ فشل تحميل: {media_url} - {e}")
            media_url = "Download Failed"

    # حفظ السطر
    data.append({
        "Bank_Name": "Bank ABC",
        "Media_Name": media_name,
        "Media_Type": media_type,
        "Media_Link": media_url or "No Media",
        "Article_Link": article_link,
        "Title": title,
        "Short_Description": description
    })


✔️ تم تحميل: About-BankABC.mp4
✔️ تم تحميل: ABC-building.mp4
✔️ تم تحميل: BankABC-IFAW2024-Ar.mp4
✔️ تم تحميل: Trade-Finance-Award-SM.mp4
✔️ تم تحميل: BankABC-Sustainability-Report-Promotion.mp4
✔️ تم تحميل: Banner-Ar-FAW.mp4
✔️ تم تحميل: Onboarding-new.mp4
✔️ تم تحميل: BankABC-JPM-Blockchain-SM-wLogo.mp4
✔️ تم تحميل: 2-TheWorldBestFinancialInnovationLabs-Global%20Finance.jpg
✔️ تم تحميل: Ila-1.mp4


In [31]:
df = pd.DataFrame(data)
df

,Bank_Name,Media_Name,Media_Type,Media_Link,Article_Link,Title,Short_Description
0,Bank ABC,About-BankABC.mp4,Video,https://www.bank-abc.com/SiteAssets/About-Bank...,https://www.bank-abc.com/ar/AboutABC/snapshot,البنك الدولي الرائد في منطقة الشرق الأوسط وشما...,​عن بنك ABC
1,Bank ABC,ABC-building.mp4,Video,https://www.bank-abc.com/SiteAssets/ABC-buildi...,https://www.bank-abc.com/ar/Media%20Relations/...,الأرباح الصافیة العائدة إلى مساھمي الشركة الأم...,​النتائج المالیة لنهاية العام٢٠٢٤​
2,Bank ABC,BankABC-IFAW2024-Ar.mp4,Video,https://www.bank-abc.com/SiteAssets/BankABC-IF...,https://www.bank-abc.com/ar/security,​​التوعية ضد الاحتيال المالي والأمن عبر الإنتر...,معًا ضد الاحتيال
3,Bank ABC,Trade-Finance-Award-SM.mp4,Video,https://www.bank-abc.com/SiteAssets/Trade-Fina...,https://www.bank-abc.com/ar/Media%20Relations/...,جائزة أفضل بنك لتمويل التجارة الدولية في الشرق...,بنك ABC يحصد​​
4,Bank ABC,BankABC-Sustainability-Report-Promotion.mp4,Video,https://www.bank-abc.com/en/SiteAssets/BankABC...,https://www.bank-abc.com/ar/Media%20Relations/...,بنك ABC يُصدر تقريره الأول لإفصاحات الاستدامة ...,الاستدامــــة
5,Bank ABC,Banner-Ar-FAW.mp4,Video,https://www.bank-abc.com/en/SiteAssets/Banner-...,https://www.bank-abc.com/ar/Media%20Relations/...,​بجائزة أفضل بنك في البحرين للعام 2023​,​فائــزون وبفخــر... للمــرة الثالثــة
6,Bank ABC,Onboarding-new.mp4,Video,https://www.bank-abc.com/SiteAssets/Onboarding...,https://www.bank-abc.com/ar/Corporate-Onboarding/,أصبح فتح الحساب أسرع وأبسط رقميا!​,التسجيل الرقمي للشركات​
7,Bank ABC,BankABC-JPM-Blockchain-SM-wLogo.mp4,Video,https://www.bank-abc.com/SiteAssets/BankABC-JP...,https://www.bank-abc.com/ar/Media%20Relations/...,​بالتعاون مع مصرف البحرين المركزي، بنك ABC يطر...,آخر الأخبـار​
8,Bank ABC,2-TheWorldBestFinancialInnovationLabs-Global%2...,Image,https://www.bank-abc.com/en/PublishingImages/S...,https://www.bank-abc.com/ar/Media%20Relations/...,اختيار مختبرات بنك ABC كأحد أفضل مختبرات الابت...,جلوبال فاينانس​
9,Bank ABC,Ila-1.mp4,Video,https://www.bank-abc.com/SiteAssets/Ila-1.mp4,https://ilabank.com/ar-BH,اختبر مفهوماً جديدًا للحساب المصرفي،​وابدأ رحل...,إلى بنك​
